# Testing Deferred Revenue in Python
Will this be easier for everyone to use than Matlab?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Step 1: Processing Base Billings Data

In [2]:
df = pd.read_excel('../data/all_billings_inputs.xlsx', sheet_name='base_billings')

In [3]:
df.sample(10)

,Document Currency,Enterprise BU Desc,Frequency,Invoice Fiscal Year Period Desc,Product Config Type,Rev Rec Category,Rule For Bill Date,Sales Type,Subscription Term,Completed Sales ( DC ),Completed Sales
12653,EUR,Creative,NaN,2016-04,MTHLY,D,Y3,DEFERRED,1,678776.78,752802.03
20407,GBP,Creative,NaN,2016-11,1Y,NaN,NaN,RECOGNIZED,0,0.00,0.00
49365,USD,Print & Publishing,NaN,2019-04,1Y,D,YQ,DEFERRED,0,509458.08,509458.08
35707,SEK,Document Cloud,NaN,2019-02,NaN,D,Y3,DEFERRED,1,43406.99,4842.24
2858,AUD,Experience Cloud,1TME,2016-04,NaN,B,NaN,DEFERRED,0,-5750.00,-4053.96
17426,EUR,Experience Cloud,ONGO,2016-09,ONORE,D,YH,DEFERRED,0,294172.94,334451.90
17764,EUR,Experience Cloud,ONGO,2017-11,1Y,D,Y1,DEFERRED,12,20913.19,24672.54
10437,DKK,Creative,NaN,2018-02,MTHLY,A,NaN,DEFERRED,1,-11076.98,-1793.14
39977,USD,Creative,ONGO,2018-09,NaN,NaN,NaN,RECOGNIZED,0,12140.00,12140.00
38385,USD,Creative,NaN,2017-12,MTHLY,NaN,NaN,DEFERRED,1,-174442.40,-174442.40


## Changing the column names early since they are inconsistent across other reports

In [4]:
df.columns

Index(['Document Currency', 'Enterprise BU Desc', 'Frequency',
       'Invoice Fiscal Year Period Desc', 'Product Config Type',
       'Rev Rec Category', 'Rule For Bill Date', 'Sales Type',
       'Subscription Term', 'Completed Sales ( DC )', 'Completed Sales'],
      dtype='object')

In [5]:
df.rename(index = str, columns = {'Document Currency': 'curr',
                                 'Enterprise BU Desc': 'BU',
                                 'Invoice Fiscal Year Period Desc': 'period',
                                 'Product Config Type': 'config',
                                 'Rev Rec Category': 'rev_req_type',
                                 'Rule For Bill Date': 'rebill_rule',
                                 'Completed Sales ( DC )': 'DC_amount',
                                 'Completed Sales': 'US_amount'}, inplace=True)

In [66]:
df.columns

Index(['curr', 'BU', 'Frequency', 'period', 'config', 'rev_req_type',
       'rebill_rule', 'Sales Type', 'Subscription Term', 'DC_amount',
       'US_amount'],
      dtype='object')

## Filter that removes any currency that has  < 10 transactions. 


In [67]:
# creates a list of the currencies and the number of transactions for each currency
vc = df['curr'].value_counts()
print(vc)

USD    11140
EUR     6940
GBP     5312
AUD     4647
JPY     4557
CHF     2306
SEK     2216
DKK     2137
NOK     1835
CAD     1362
HKD      485
BRL      464
RUB      461
KRW      247
COP      210
CLP      209
ARS      208
SGD      199
PEN      190
INR      178
PHP      151
TWD      148
THB      144
MYR      137
IDR      132
NZD       58
ILS       35
TRY       27
Name: curr, dtype: int64


In [68]:
# Create variable that is true if the number of transaction is greater than 10, false otherwise
keep_these = vc.values > 10
# filtering only currencies that were greater than 10
keep_curr = vc[keep_these]
a = keep_curr.index
# filtering the dataframe to remove any of teh currencies not in our list
df = df[df['curr'].isin(a)]

## Just keeping track of the currencies we removed in our model_dict data structure

In [69]:
remove_these = vc[vc.values <= 10].index
model_dict = {'curr_removed': list(vc[remove_these].index)}
delete_curr = list(remove_these)

## The FX database does not have information on the following currencies
 - AED (United Arab Emirates Dirham)
 - BMD (Bermudan Dollar)
 - MXP (Mexican Peso)
 - TRY (Turkish Lira)

In [70]:
if 'TRY' not in model_dict:
    model_dict['curr_removed'].append('TRY')
    delete_curr.append('TRY')
    a = a.drop('TRY')
    
print('Model dictionary', model_dict)
print('Deleted Currencies', delete_curr)

Model dictionary {'curr_removed': ['TRY']}
Deleted Currencies ['TRY']


In [71]:
print("---Removing infrequent currencies from billings history---")
print('Total number of currencies in the base billings file: ', len(vc))
if len(model_dict['curr_removed'])==0:
    print('No currencies were removed, all contained 10 or more billings')
    print('Currencies in the base billings file')
    for item in a:
        print(a[item], end = " ")
else:
    print('\n Currencies were removed: ', len(model_dict['curr_removed']))

    for item in remove_these:
        print(item, ', ', end="")
        
    print("\n\n{} Remaining currencies: ".format(len(a)))
    for item in a:
        print(item, ', ', end="")

---Removing infrequent currencies from billings history---
Total number of currencies in the base billings file:  28

 Currencies were removed:  1


27 Remaining currencies: 
USD , EUR , GBP , AUD , JPY , CHF , SEK , DKK , NOK , CAD , HKD , BRL , RUB , KRW , COP , CLP , ARS , SGD , PEN , INR , PHP , TWD , THB , MYR , IDR , NZD , ILS , 

# Removing any of the values that are zero

In [72]:
print('This is the length of the dataframe before removing zeros: ', len(df))
df = df[df['DC_amount']!=0]
print('This is the length of the dataframe after removing zeros: ', len(df))

This is the length of the dataframe before removing zeros:  46135
This is the length of the dataframe after removing zeros:  46135


In [73]:
df.sample(10)

,curr,BU,Frequency,period,config,rev_req_type,rebill_rule,Sales Type,Subscription Term,DC_amount,US_amount
38468,USD,Creative,NaN,2018-03,2V,NaN,NaN,RECOGNIZED,0,49.00,49.00
12536,EUR,Creative,NaN,2015-12,NaN,A,NaN,DEFERRED,12,-44.71,-49.19
37122,THB,Document Cloud,NaN,2019-11,NaN,D,NaN,DEFERRED,1,-534.00,-17.56
48471,USD,Print & Publishing,NaN,2015-09,NaN,A,NaN,DEFERRED,1,131047.18,131047.18
6818,CAD,Experience Cloud,ONGO,2018-05,NaN,NaN,NaN,RECOGNIZED,0,171032.64,133417.56
24716,GBP,LiveCycle,NaN,2016-12,NaN,NaN,NaN,RECOGNIZED,0,3760.57,4645.16
22581,GBP,Experience Cloud,NaN,2016-12,1Y,D,YA,DEFERRED,0,15044.54,18754.97
41690,USD,Document Cloud,NaN,2019-12,1Y,D,Y3,DEFERRED,1,3487602.61,3487602.61
4608,AUD,Other Solutions,ONGO,2016-11,OUNIV,A,NaN,DEFERRED,0,2560.00,1960.70
46686,USD,Experience Cloud,ONGO,2018-02,1Y,D,YH,DEFERRED,0,774623.00,774623.00


## Clearing out the Non-Revenue billings from the file
 - No Idea what these are

In [74]:
df["Sales Type"].value_counts()

DEFERRED       37628
RECOGNIZED      7183
PRO-SVC-INV     1323
Name: Sales Type, dtype: int64

In [75]:
print('Length of the dataframe before removing non-revenue billings: ', len(df))
df = df[df['Sales Type']!='NON-REV']
print('Length of the dataframe after removing non-revenue billings:  ', len(df))


Length of the dataframe before removing non-revenue billings:  46135
Length of the dataframe after removing non-revenue billings:   46135


## Starting to group the revenue by period, industry, etc

Attempting to group by the following categories
 - currency
 - period
 - sale type
 
May need to process the data differently with the deferred billings so we will start with the recognized and then the service billings

# DOING THIS ALL IN PANDAS WITH SPLIT APPLY COMBINE on Sales Type 


In [76]:
# First split the data into three dataframes
# Recognized billings
rec = df[df['Sales Type']=='RECOGNIZED']
svc = df[df['Sales Type']=='PRO-SVC-INV']
dfr = df[df['Sales Type']=='DEFERRED']

# NOW WORKING ON THE BILLINGS

### Recognized Revenue

In [77]:
rec.sample(10)

,curr,BU,Frequency,period,config,rev_req_type,rebill_rule,Sales Type,Subscription Term,DC_amount,US_amount
31434,JPY,Print & Publishing,NaN,2020-04,NaN,NaN,NaN,RECOGNIZED,0,8.894203e+06,81517.75
25518,GBP,Print & Publishing,NaN,2019-04,NaN,NaN,NaN,RECOGNIZED,0,6.921080e+04,91457.54
16765,EUR,Experience Cloud,ACTL,2017-09,ONORE,NaN,NaN,RECOGNIZED,0,8.913630e+05,1053468.95
2831,AUD,Experience Cloud,1TME,2016-01,NaN,B,NaN,RECOGNIZED,0,5.000000e+02,362.12
27016,JPY,Creative,NaN,2016-09,UPG1,NaN,NaN,RECOGNIZED,0,3.750000e+05,3693.90
14188,EUR,Document Cloud,NaN,2016-03,NaN,NaN,NaN,RECOGNIZED,0,-3.240000e+02,-356.50
4546,AUD,LiveCycle,NaN,2017-05,NaN,NaN,NaN,RECOGNIZED,0,3.149250e+03,2383.30
27750,JPY,Creative,NaN,2019-09,NaN,NaN,NaN,RECOGNIZED,0,1.076450e+08,1002710.70
27280,JPY,Creative,NaN,2017-10,IDRT,NaN,NaN,RECOGNIZED,0,1.329760e+06,12047.62
16089,EUR,Experience Cloud,1TME,2017-05,NaN,NaN,NaN,RECOGNIZED,0,6.273153e+04,67404.14


In [78]:
# testing groupby object
gb_rec = rec.groupby(['curr', 'BU', 'period'], as_index=False).sum()

In [79]:
# the Subscription term hangs around. We are dropping that here
gb_rec.drop(labels='Subscription Term', axis=1,inplace =True)

In [80]:
gb_rec.sample(10)

,curr,BU,period,DC_amount,US_amount
2634,SEK,Document Cloud,2016-10,3020.00,353.55
2514,PHP,Creative,2020-04,36423.00,714.67
1716,GBP,Other Solutions,2019-03,-750.47,-981.49
1351,EUR,Other Solutions,2019-02,-13305.80,-15158.98
260,AUD,Other Solutions,2018-10,-5292.01,-3784.49
199,AUD,Experience Cloud,2020-02,307712.86,212006.26
860,DKK,Document Cloud,2017-06,590.00,88.84
419,CAD,Experience Cloud,2019-07,997229.89,749832.70
2496,NZD,Experience Cloud,2020-03,472254.75,308563.35
3235,USD,Other Solutions,2016-05,234938.11,234938.11


## Now doing this for the service billings

In [81]:
# testing groupby object
gb_svc = svc.groupby(['curr', 'BU', 'period'], as_index=False).sum()

In [82]:
gb_svc.drop(labels='Subscription Term', axis=1,inplace =True)
gb_svc.head(5)

,curr,BU,period,DC_amount,US_amount
0,AUD,Experience Cloud,2015-01,25075.00,21084.51
1,AUD,Experience Cloud,2015-02,-4750.00,-4220.74
2,AUD,Experience Cloud,2015-03,424271.75,337822.37
3,AUD,Experience Cloud,2015-04,297925.00,229116.23
4,AUD,Experience Cloud,2015-05,316894.82,237310.85


# NOW WORKING ON DEFERRED BILLINGS

## Type B billings are service agreements that will have invoices submitted before the billings are reclassified to revenue. If no invoices are assigned to the billings, the billings become revenue in 12 months

In [83]:
# filter out the type B first then do a group_by
dfr_b = dfr[dfr['rev_req_type']=='B']

In [84]:
gb_b = dfr_b.groupby(['curr', 'BU', 'period'], as_index=False).sum()

In [85]:
gb_b.drop(labels='Subscription Term', axis=1, inplace=True)

In [86]:
gb_b.head(10)

,curr,BU,period,DC_amount,US_amount
0,AUD,Creative,2017-10,8544.0,6837.59
1,AUD,DX Other,2017-04,13205.0,10082.02
2,AUD,DX Other,2017-08,27400.0,21259.93
3,AUD,DX Other,2018-07,6635.4,4908.74
4,AUD,DX Other,2018-08,7535.0,5561.43
5,AUD,Document Cloud,2017-12,3283.2,2494.58
6,AUD,Document Cloud,2018-06,3425.0,2592.18
7,AUD,Document Cloud,2018-12,3425.0,2483.47
8,AUD,Document Cloud,2019-03,4375.0,3122.00
9,AUD,Experience Cloud,2015-01,157510.0,131675.74


In [87]:
print('length of deferred billings : ', len(dfr))
print('length of the type B billings: ', len(dfr_b))

length of deferred billings :  37628
length of the type B billings:  1434


## Now Type A Billings
These billings are on a billing plan. The product config tells us how long before they renew

 - '2Y' = 24 months
 - '1Y' = 12 months
 - 'MTHLY' = 1 month
 
NOTE: There are also other fields in the 'Product Configtype ID' field that do not map well to a rebill period.
To fix this, we need to load up a different file and determine the length of the sales contract (type A no config)
 

In [88]:
# filtering just the type A billings
dfr_a = dfr[dfr['rev_req_type']=='A']

In [89]:
gb_a = dfr_a.groupby(['curr', 'BU', 'period',
                     'config'], as_index=False).sum()
gb_a.drop(labels='Subscription Term', axis=1, inplace = True)

In [90]:
gb_a.sample(10)

,curr,BU,period,config,DC_amount,US_amount
3102,JPY,Experience Cloud,2016-12,1Y,9.312991e+07,835317.60
2868,JPY,Creative,2019-11,1Y,1.495522e+09,10805918.28
446,AUD,LiveCycle,2016-01,1Y,1.251022e+04,9053.15
59,AUD,Creative,2016-08,MTHLY,-1.151986e+04,-8665.64
1507,EUR,Creative,2019-09,1Y,3.189519e+07,35066165.02
3631,SEK,Document Cloud,2016-02,1Y,-1.999630e+03,-234.11
22,AUD,Creative,2015-08,2Y,2.316620e+03,1779.90
173,AUD,Document Cloud,2015-02,2Y,8.710500e+03,7114.25
4440,USD,Experience Cloud,2019-09,1Y,7.865042e+06,7865042.21
2941,JPY,Document Cloud,2016-05,MTHLY,-3.506700e+04,-317.49


In [91]:
gb_a['config'].value_counts()

1Y       2418
MTHLY     950
2Y        875
OUNIV     231
OCONS     106
3Y        101
ONORE      32
1V          1
OENSV       1
Name: config, dtype: int64

### Below is just a check to see how large the billing types are across all periods

In [92]:
gb_a_config = gb_a.groupby(['config'], as_index=False).sum()
gb_a_config

,config,DC_amount,US_amount
0,1V,6.503000e+03,6.503000e+03
1,1Y,1.198135e+11,9.655797e+09
2,2Y,2.470307e+08,6.719101e+07
3,3Y,9.380464e+08,4.051346e+06
4,MTHLY,1.381940e+08,3.985406e+07
5,OCONS,3.903233e+07,1.028252e+07
6,OENSV,-1.200000e+02,-1.200000e+02
7,ONORE,2.350904e+06,2.015485e+06
8,OUNIV,4.283761e+06,3.473767e+06


### These 'OCONS', 'ONORE' and 'OUNIV' data types are not actual product config IDs so we have to get them from a different data file. We are excluding these types below.

In [93]:
config_list = ['1Y', '2Y', '3Y', 'MTHLY']
test1 = gb_a[gb_a['config'].isin(config_list)]


In [94]:
test1.sample(5)

,curr,BU,period,config,DC_amount,US_amount
2400,GBP,Document Cloud,2020-02,1Y,1114507.98,1455902.46
3169,JPY,LiveCycle,2017-05,1Y,22323734.00,203249.56
1151,DKK,Creative,2016-09,MTHLY,-11470.57,-1719.26
2503,GBP,LiveCycle,2017-04,1Y,10427.45,12808.46
2842,JPY,Creative,2018-11,MTHLY,-2115492.00,-18838.98


## For now, lets just split this into gb_a_1Y, gb_a_2Y, gb_a_3y, gb_a_1M


In [95]:
gb_a_1Y = test1[test1['config']=='1Y']
gb_a_2Y = test1[test1['config']=='2Y']
gb_a_3Y = test1[test1['config']=='3Y']
gb_a_1M = test1[test1['config']=='MTHLY']

In [96]:
print('this is the lenght of type A 1M billings: ', len(gb_a_1M))
print('this is the lenght of type A 1Y billings: ', len(gb_a_1Y))
print('this is the lenght of type A 2Y billings: ', len(gb_a_2Y))
print('this is the lenght of type A 3Y billings: ', len(gb_a_3Y))

this is the lenght of type A 1M billings:  950
this is the lenght of type A 1Y billings:  2418
this is the lenght of type A 2Y billings:  875
this is the lenght of type A 3Y billings:  101


# TYPE D billings
These billings have a field 'Rule For Bill Date' that determines when new billings will occur
 - Monthly [Y1, Y2, Y3, Y5]
 - Quarterly [YQ]
 - Every 4 months [YT]
 - Semi-annual [YH]
 - Annual [YA, YC]
 - Every 2 years - [Y4]
 
 We also need to track the type D billings that do not have a 'Rule for Bill Date'

In [97]:
# for now just do a groupby on the type
# filtering just the type A billings
dfr_d = dfr[dfr['rev_req_type']=='D']

In [98]:
gb_d = dfr_d.groupby(['curr', 'BU', 'period',
                     'rebill_rule'], as_index=False).sum()
gb_d.drop(labels='Subscription Term', axis=1, inplace = True)

In [99]:
gb_d_mthly = gb_d[gb_d['rebill_rule'].isin(['Y1', 'Y2', 'Y3', 'Y5'])]
gb_d_qtrly = gb_d[gb_d['rebill_rule']=='YQ']
gb_d_four_mths = gb_d[gb_d['rebill_rule']=='YT']
gb_d_semi_ann = gb_d[gb_d['rebill_rule']=='YH']
gb_d_annual = gb_d[gb_d['rebill_rule'].isin(['YA', 'YC'])]
gb_d_two_yrs = gb_d[gb_d['rebill_rule']=='Y4']

In [100]:
gb_d['rebill_rule'].value_counts()

YA    2952
Y3    2375
YQ     991
YH     547
Y1     491
YC     490
Y2     153
Y4      57
Y7      31
YM      23
YX      12
YT      10
YY      10
Name: rebill_rule, dtype: int64

# Asking Michelle to follow up on the rebill rules I don't know here

### Note: There is one type D billings that is listed as 'BT' I don't know what this means, but it was a $180 EUR payment from January 2017, so we will ignore

In [101]:
what_is_YT = gb_d[gb_d['rebill_rule']=='YT']
what_is_YT.head(10)

,curr,BU,period,rebill_rule,DC_amount,US_amount
6985,USD,DX Other,2016-12,YT,13440.00,13440.00
6999,USD,DX Other,2017-04,YT,13440.00,13440.00
7011,USD,DX Other,2017-08,YT,13440.00,13440.00
7436,USD,Experience Cloud,2015-02,YT,612916.60,612916.60
7444,USD,Experience Cloud,2015-03,YT,698888.89,698888.89
7465,USD,Experience Cloud,2015-06,YT,11666.60,11666.60
7480,USD,Experience Cloud,2015-08,YT,549861.11,549861.11
7495,USD,Experience Cloud,2015-10,YT,11666.60,11666.60
7518,USD,Experience Cloud,2016-01,YT,612916.60,612916.60
7526,USD,Experience Cloud,2016-02,YT,-601250.00,-601250.00


## Note: There are only 8 payments (back from 2017) that paid on a 4 month basis. 
We will ignore these as well.

In [102]:
print('Length of monthly', len(gb_d_mthly))
print('Length of quarterly', len(gb_d_qtrly))
print('Length of four months', len(gb_d_four_mths))
print('Length of semi ann', len(gb_d_semi_ann))
print('Length of annual', len(gb_d_annual))
print('Length of two years', len(gb_d_two_yrs))

Length of monthly 3019
Length of quarterly 991
Length of four months 10
Length of semi ann 547
Length of annual 3442
Length of two years 57


## NOW WE NEED TO BUILD A DATAFRAME THAT INTEGRATES THIS DATA

- We will have the following descriptive fields
   - Invoicing Fiscal Year-Period
   - Document Currency
   - Enterprise BU

- We will have the following fields based on rebilling rule
   - Recognized
   - Service
   - Monthly
   - Quarterly
   - Annual
   - Two Years
   - Three Years

In [126]:
# We need to do it this way when we get to a .py file!
list_df = [gb_rec, gb_svc, gb_b,
        gb_a_1M,    gb_a_1Y,    gb_a_2Y,       gb_a_3Y, 
        gb_d_mthly, gb_d_qtrly, gb_d_semi_ann, gb_d_annual, gb_d_two_yrs]

list_columns = ['recognized', 'service', 'deferred_B', 
    'deferred_1M_a', 'deferred_1Y_a', 'deferred_2Y_a', 'deferred_3Y_a',
    'deferred_1M_d', 'deferred_3M_d', 'deferred_6M_d', 'deferred_1Y_d', 'deferred_2Y_d']


In [127]:
def merge_new_dataframe(old_df, new_df, new_column):
    df_merged = pd.merge(old_df, new_df, how='outer', 
                     left_on=['curr', 'BU', 'period'],
                    right_on=['curr', 'BU', 'period'])
    df_merged.rename(index=str, columns={'DC_amount': new_column+'_DC', 'US_amount': new_column+'_US'}, inplace=True)
    
    #need to drop the product configtype id for merges where the new_df is of type A
    config_str = 'config'
    rule_str = 'rebill_rule'
    if config_str in df_merged.columns:
        df_merged.drop(columns=['config'], inplace=True)
    
    if rule_str in df_merged.columns:
        df_merged.drop(columns=['rebill_rule'], inplace=True)
        
    return df_merged

In [128]:
def merge_all_dataframes(list_df, list_columns):
    for i, df in enumerate(list_df):
        print('This is i:', i)
        #print("This is the df: ", df.head())
        print('referencing the column: ', list_columns[i])

        if i==0:
            df_merged = list_df[0].copy()
            df_merged.rename(index=str, columns={'DC_amount': list_columns[i]+'_DC', 
                                                 'US_amount': list_columns[i]+'_US'}, inplace=True)
        else:
            df_merged = merge_new_dataframe(df_merged, df, list_columns[i])

    return df_merged
    

In [134]:
def clean_df_columns(df):
    
    # clean up NaNs before adding 
    df = df.fillna(value=0)
    
    # DC columns first
    # Monthly
    df['deferred_1M_DC'] = df['deferred_1M_a_DC']+df['deferred_1M_d_DC']
    df.drop(labels=['deferred_1M_a_DC', 'deferred_1M_d_DC'], axis=1, inplace=True)
    
    # Annual
    df['deferred_1Y_DC'] = df['deferred_1Y_a_DC']+df['deferred_1Y_d_DC']
    df.drop(labels=['deferred_1Y_a_DC', 'deferred_1Y_d_DC'], axis=1, inplace=True)
    
    # Two-Year
    df['deferred_2Y_DC'] = df['deferred_2Y_a_DC']+df['deferred_2Y_d_DC']
    df.drop(labels=['deferred_2Y_a_DC', 'deferred_2Y_d_DC'], axis=1, inplace=True)
    
    # renaming 3Y, 3M and 6M
    df.rename(index=str, columns = {'deferred_3Y_a_DC':'deferred_3Y_DC', 'deferred_3M_d_DC':'deferred_3M_DC', 
                               'deferred_6M_d_DC': 'deferred_6M_DC'}, inplace=True)

    # US columns
    # Monthly
    df['deferred_1M_US'] = df['deferred_1M_a_US']+df['deferred_1M_d_US']
    df.drop(labels=['deferred_1M_a_US', 'deferred_1M_d_US'], axis=1, inplace=True)
    
    # Annual
    df['deferred_1Y_US'] = df['deferred_1Y_a_US']+df['deferred_1Y_d_US']
    df.drop(labels=['deferred_1Y_a_US', 'deferred_1Y_d_US'], axis=1, inplace=True)
    
    # Two-Year
    df['deferred_2Y_US'] = df['deferred_2Y_a_US']+df['deferred_2Y_d_US']
    df.drop(labels=['deferred_2Y_a_US', 'deferred_2Y_d_US'], axis=1, inplace=True)
    
    # renaming 3Y, 3M and 6M
    df.rename(index=str, columns = {'deferred_3Y_a_US':'deferred_3Y_US', 'deferred_3M_d_US':'deferred_3M_US', 
                               'deferred_6M_d_US': 'deferred_6M_US'}, inplace=True)

    
    #cleaning up the longer column names
    df.rename(index=str, columns = {'curr': 'curr',
                               'BU':'BU',
                               'period':'period'}, inplace=True)
    
    return df

In [129]:
df = merge_all_dataframes(list_df, list_columns)

This is i: 0
referencing the column:  recognized
This is i: 1
referencing the column:  service
This is i: 2
referencing the column:  deferred_B
This is i: 3
referencing the column:  deferred_1M_a
This is i: 4
referencing the column:  deferred_1Y_a
This is i: 5
referencing the column:  deferred_2Y_a
This is i: 6
referencing the column:  deferred_3Y_a
This is i: 7
referencing the column:  deferred_1M_d
This is i: 8
referencing the column:  deferred_3M_d
This is i: 9
referencing the column:  deferred_6M_d
This is i: 10
referencing the column:  deferred_1Y_d
This is i: 11
referencing the column:  deferred_2Y_d


In [135]:
df = clean_df_columns(df)

In [136]:
df.sample(10)

,curr,BU,period,recognized_DC,recognized_US,service_DC,service_US,deferred_B_DC,deferred_B_US,deferred_3Y_DC,...,deferred_3M_DC,deferred_3M_US,deferred_6M_DC,deferred_6M_US,deferred_1M_DC,deferred_1Y_DC,deferred_2Y_DC,deferred_1M_US,deferred_1Y_US,deferred_2Y_US
2814,NOK,Creative,2018-12,204928.00,24343.52,0.00,0.00,0.00,0.00,0.0,...,0.00,0.00,0.00,0.00,16932131.76,6870577.67,0.00,2007555.88,812401.82,0.00
4664,CAD,Creative,2017-12,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.00,0.00,0.00,0.00,0.00,2702101.83,0.00,0.00,2120443.17,0.00
4383,USD,Other Solutions,2016-10,-2939.89,-2939.89,0.00,0.00,0.00,0.00,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2922,NOK,Experience Cloud,2017-09,133897.09,16850.76,128073.50,16308.88,0.00,0.00,0.0,...,1752617.50,221544.18,0.00,0.00,0.00,800202.00,0.00,0.00,100729.43,0.00
3351,SEK,Other Solutions,2017-07,-465.66,-53.61,0.00,0.00,0.00,0.00,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3492,USD,Creative,2015-06,31105538.88,31105538.88,179021.88,179021.88,17700.16,17700.16,0.0,...,714356.34,714356.34,92888.32,92888.32,297417.73,54049378.76,132701.06,297417.73,53771030.56,132701.06
3551,USD,Creative,2016-06,11055078.61,11055078.61,10125.00,10125.00,-29690.00,-29690.00,0.0,...,672870.27,672870.27,218150.47,218150.47,561926.46,77943969.56,97736.20,561926.46,76318051.66,97736.20
555,CAD,LiveCycle,2017-10,961.35,781.84,0.00,0.00,0.00,0.00,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
70,AUD,Creative,2019-03,164031.92,117030.66,0.00,0.00,0.00,0.00,0.0,...,0.00,0.00,0.00,0.00,19522071.17,12720059.66,12.00,13954587.91,9049823.90,8.57
5134,THB,Document Cloud,2019-10,0.00,0.00,0.00,0.00,0.00,0.00,0.0,...,0.00,0.00,0.00,0.00,23123.00,81451.00,0.00,756.36,2663.51,0.00


In [137]:
df.columns


Index(['curr', 'BU', 'period', 'recognized_DC', 'recognized_US', 'service_DC',
       'service_US', 'deferred_B_DC', 'deferred_B_US', 'deferred_3Y_DC',
       'deferred_3Y_US', 'deferred_3M_DC', 'deferred_3M_US', 'deferred_6M_DC',
       'deferred_6M_US', 'deferred_1M_DC', 'deferred_1Y_DC', 'deferred_2Y_DC',
       'deferred_1M_US', 'deferred_1Y_US', 'deferred_2Y_US'],
      dtype='object')

In [ ]:
# Make this a function to be cleaned up somehow
del dfr
del dfr_a
del dfr_b
del dfr_d
del gb_a
del gb_a_1M
del gb_a_1Y
del gb_a_2Y
del gb_a_3Y
del gb_b, 
del gb_d
del gb_svc, gb_rec, gb_d_two_yrs
del gb_d_four_mths, gb_d_qtrly, gb_d_semi_ann


In [ ]:
df.head(20)

In [ ]:
gb_rec

In [ ]:
whos

# Now working on the ZCC billings

These billings are type D billings that did not populate the rebill_rule field of the database.

They have a 'sales document type' = 'ZCC"

The billings themselves are being created from a tableau report that looks for additions to the deferred revenue waterfall based on billings of type D and have a sales document type of ZCC

In [ ]:
whos

In [ ]:
df.sample(10)

# TO BE DONE:

1. Clean up the type F billings (at least check to see if they are necessary)
2. Make a function to delete all intermediate dataframes
3. Add type A no config function
4. Add type D ZCC billings

5. Work on the forecast part of this

6. Load up FX rates

In [ ]:
# Adobe financial calendar
df_cal = pd.read_excel('../data/old/ADOBE_FINANCIAL_CALENDAR.xlsx', 'ADBE_cal')

In [ ]:
df_cal.head()

## Working on Type A billings with no configuration

 - this is included in the all_billings_inputs file on the 'type_A_no_config' sheet

## ZCC Billings have been included in the base billings file

In [ ]:
df_ZCC = pd.read_excel('../data/all_billings_inputs.xlsx', sheet_name='type_A_no_config')

In [ ]:

# clean up NaNs before adding 
test_merge = test_merge.fillna(value=0)

# Monthly
test_merge['deferred_1M'] = test_merge['deferred_1M_x']+test_merge['deferred_1M_y']
test_merge.drop(labels=['deferred_1M_x', 'deferred_1M_x'], axis=1, inplace=True)

# Quarterly
test_merge['deferred_3M'] = test_merge['deferred_3M_x']+test_merge['deferred_3M_y']
test_merge.drop(labels=['deferred_3M_x', 'deferred_3M_y'], axis=1, inplace=True)

# Semi-Annual
test_merge['deferred_6M'] = test_merge['deferred_6M_x']+test_merge['deferred_6M_y']
test_merge.drop(labels=['deferred_6M_x', 'deferred_6M_y'], axis=1, inplace=True)

# Annual
test_merge['deferred_1Y'] = test_merge['deferred_1Y_x']+test_merge['deferred_1Y_y']
test_merge.drop(labels=['deferred_1Y_x', 'deferred_1Y_y'], axis=1, inplace=True)

# 2-Years
test_merge['deferred_2Y'] = test_merge['deferred_2Y_x']+test_merge['deferred_2Y_y']
test_merge.drop(labels=['deferred_2Y_x', 'deferred_2Y_y'], axis=1, inplace=True)





In [ ]:
test_merge.columns

In [ ]:
test_merge.sample(20)

In [ ]:
## Looks like the type ZCC billings are complete

In [ ]:
# renaming the main dataframe as df
df = test_merge.copy()

# Type A No Config Type Billings

This file contains type A billings that have a revenue contract start date and end date. We need to map these into the terms of our dataframe.

### Steps:
1. Rename the columns
2. This file has entries for pennies. Need to clear out anything less than $10 in absolute value
3. Determine the length of time between start date and end date
4. Group this dataframe by currency, period and BU
5. Merge this final dataframe with the larger dataframe

## NOTE: This file contains two different start date and end date columns. We need to look at all of them

In [ ]:
df_A = pd.read_excel('../data/old/type_A_no_config.xlsx', 'DC')

In [ ]:
df_A.sample(10)

In [ ]:
df_A.columns

In [ ]:
df_A.rename(index=str, columns={'Document Currency':'currency', 
                               'Enterprise Bu':'BU',
                               'Invoicing Fiscal Year-Period Desc':'period',
                               'Rev Rec Contract End Date Hdr':'end_date_1',
                               'Rev Rec Contract End Date Item':'end_date_2',
                               'Rev Rec Contract Start Date Hdr': 'start_date_1',
                               'Rev Rec Contract Start Date Item': 'start_date_2',
                               'Completed Sales Doc Currency':'amount'}, inplace=True)

df_A.drop(columns='Product Configtype ID', inplace=True)

In [ ]:
df_A.columns

### Dealing with the duplicate dates by taking a max

In [ ]:
df_A.head()

In [ ]:
df_A['start_date_str'] = df_A[['start_date_1','start_date_2']].max(axis=1).astype(int).astype(str)
df_A['end_date_str'] = df_A[['end_date_1','end_date_2']].max(axis=1).astype(int).astype(str)

In [ ]:
df_A.sample(10)

In [ ]:
df_A['start_date'] = pd.to_datetime(df_A['start_date_str'])
df_A['end_date'] = pd.to_datetime(df_A['end_date_str'])

In [ ]:
df_A.drop(labels=['end_date_1', 'end_date_2', 'start_date_1', 'start_date_2',
                  'start_date_str', 'end_date_str'], axis=1, inplace=True)

In [ ]:
df_A['month_interval']=(df_A['end_date']-df_A['start_date'])

In [ ]:
df_A.sample(10)

In [ ]:
df_A['months']= (df_A['month_interval']/ np.timedelta64(1,'M')).round(0)

### Now I need to map the months into the different integers in my dataframe


In [ ]:
list_rebills = [1, 3, 6, 12, 24, 36]
temp_rebill = np.zeros_like(df_A['months'])
for i in range(len(df_A)):
    temp_rebill[i] = min(list_rebills, key=lambda x:abs(x-df_A['months'][i]))
df_A['rebill_months']=temp_rebill


In [ ]:
df_A.sample(10)

In [ ]:
plt.scatter(df_A['months'], df_A['rebill_months'])

## Grouping the dataframe by rebill_months

In [ ]:
# drop what we dont need
df_A.drop(columns = ['start_date', 'end_date', 'month_interval', 'months'], axis=1, inplace=True)

In [ ]:
df_A.head()

In [ ]:
#gb_A = df+A.groupby(['curr', 'BU', 'period'], as_index=False).sum()

In [ ]:
#medals = df.pivot_table('no of medals', ['Year', 'Country'], 'medal')
temp = df_A.pivot_table('amount', ['currency', 'BU', 'period'], 'rebill_months')

In [ ]:
temp

In [ ]:
temp.columns[5]

In [ ]:
temp.fillna(0)

In [ ]:
temp.rename(index=str, columns = {''})

In [ ]:
whos